In [ ]:
from __future__ import print_function, division
from future.utils import iteritems
from builtins import range, input

import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime

import os
import sys
sys.path.append(os.path.abspath('..'))
#from rnn_class.util import get_wikipedia_data
#from rnn_class.brown import get_sentences_with_word2idx_limit_vocab,get_sentences_with_word2idx

#from markov import get_bigram_probs

if __name__ == '__main__':
    sentences,word2idx = get_sentences_with_word2idx_limit_vocab(2000)
    V =  len(word2idx)
    print("Vocab size:", V)
    start_idx = word2idx['START']
    end_idx = word2idx['END']
    bigram_probs = get_bigram_probs(sentences,V,start_idx,end_idx,smoothing=0.1)
    D=100
    W1 = np.random.randn(V,D)/np.sqrt(V)
    W2 = np.random.randn(D,V)/np.sqrt(D)

    losses = []
    epochs = 1
    lr = 1e-1

def softmax(a):
    a = a - a.max()
    exp_a = np.exp(a)
    return exp_a / exp_a.sum(axis=1, keepdims = True)
W_bigram = np.log(bigram_probs)
bigram_losses = []

t0 = datetime.now()
for epoch in range(epochs):
    random.shuffle(sentences)
    j = 0
    for sentence in sentences:
        sentence = [start_idx] + sentence +[end_idx]
        n = len(sentence)
      #  inputs = np.zeros((n-1,V))
      #  targets  = np.zeros((n-1,V))
      #  inputs[np.arange(n-1),sentence[:n-1]] = 1
      #  targets[np.arange(n-1),sentence[1:]] = 1
        inputs = sentence[:n-1]
        targets = sentence[1:]
        #hidden = np.tanh(inputs.dot(W1))
        hidden = np.tanh(W1[inputs])
        predictions = softmax(hidden.dot(W2))
        loss = -np.sum(np.log(predictions[np.arange(n-1),targets]))/(n-1)
        losses.append(loss)
        doutput = predictions
        doutput[np.arange(n-1),targets] -= 1
        W2 = W2 - lr * hidden.T.dot(doutput)
        dhidden = doutput.dot(W2.T)*(1-hidden * hidden )
     # predictions = softmax(hidden.dot(W2))
     # W2 = W2 - lr * hidden.T.dot(predictions -  targets)
     # dhidden = (predictions - targets).dot(W2.T) * (1- hidden * hidden)
     # W1 =  W1 - lr * inputs.T.dot(dhidden)


     #loss = -np.sum(targets* np.log(predictions)) / (n-1)
     #losses.append(loss)
###addedbelow lines##
        i = 0
        for w in inputs:
            W1[w] = W1[w] - lr*dhidden[i]
            i +=1
            
##till here

if epoch == 0:
    bigram_predictions = softmax((W_bigram[inputs])
    bigram_loss = -np.sum(np.log(bigram_predictions[np.arange(n-1),targets]))/(n-1)
    #bigram_predictions = softmax(inputs.dot(W_bigram))
    #bigram_loss = -np.sum(targets * np.log(bigram_predictions))/(n-1)                             
    bigram_losses.append(bigram_loss)

if j % 100 ==0:
    print("epoch:","sentence: %s /%s" %(j, len(sentences)), "loss:",loss)
j +=1

print("elaped time training:", datetime.now() - t0)
plt.plot(losses)

avg_bigram_loss = np.mean(bigram_losses)
print("avg_bigram_loss:", avg_bigram_loss)
plt.axhline(y=avg_bigram_loss,color = 'r',linestyle='-')

def smoothed_loss(x,decay=0.99):
    y = np.zeros(len(x))
    last = 0
    for t in range(len(x)):
        z = decay * last + (1- decay)* x[t]
        y[t] = z /(1-decay **(t +1))
        last = z
    return y
plt.plot(smoothed_loss(losses))
plt.show()

plt.subplot(1,2,1)
plt.title("logistic model")
plt.imshow(softmax(W2))
plt.subplot(1,2,2)
plt.title("Bigram probs")
plt.imshow(bigram_probs)
plt.show() 

In [ ]:
import numpy as np
def get_bigram_probs(sentences,V, start_idx, end_idx, smoothing=1):
    bigram_probs = np.ones((V,V)) * smoothing
    for sentence in sentences:
        for i in range(len(sentence)):
            if i ==0:
                bigram_probs[start_idx, sentence[i]] +=1
            else:
                bigram_probs[sentence[i-1],sentence[i]] +=1

            if i == len(sentence) - 1:
                bigram_probs[sentence[i], end_idx] +=1
    bigram_probs /= bigram_probs.sum(axis=1,keepdims = True)
    return bigram_probs

if __name__ == '__main__':
    sentences,word2idx = get_sentences_with_word2idx_limit_vocab(10000)
V = len(word2idx)
print("Vocab size:" , V)

start_idx = word2idx['START']
end_idx = word2idx['END']
bigram_probs = get_bigram_probs(sentences, V ,start_idx, end_idx, smoothing= 0.1)

In [ ]:
from __future__ import print_function,division
from builtins import range
from nltk.corpus import brown
import operator
import numpy as np

KEEP_WORDS = set([
'king','man','queen','woman',
'italy','rome','france','paris',
'london','britain','england',])

def get_sentences():
    return brown.sents()

def get_sentences_with_word2idx():
    sentences = get_sentences()
    indexed_sentences = []
    i = 2
    word2idx = {'START':0,'END': 1}
    for sentence in sentences:
        indexed_sentence = []
        for token in sentence:
            token = token.lower()
            if token not in word2idx:
                word2idx[token] = i
                i +=1
            indexed_sentence.append(word2idx[token])
        indexed_sentences.append(indexed_sentence)
    print('vocab size:',i)
    return indexed_sentences,word2idx

                
def get_sentences_with_word2idx_limit_vocab(n_vocab=2000,keep_words=KEEP_WORDS):
    sentences = get_sentences()
    indexed_sentences = []
    i = 2
    word2idx = {'START':0,'END': 1}
    idx2word = ['START','END']
    word_idx_count = {
      0: float('inf'),
      1: float('inf'),
    }
    for sentence in sentences:
        indexed_sentence = []
        for token in sentence:
            token = token.lower()
            if token not in word2idx:
                idx2word.append(token)
                word2idx[token] = i
                i += 1
            idx = word2idx[token]
            word_idx_count[idx] = word_idx_count.get(idx,0) + 1
            indexed_sentence.append(idx)
        indexed_sentences.append(indexed_sentence)
    for word in keep_words:
        word_idx_count[word2idx[word]] = float('inf')
        # sort technique operator.itemgetter(1)
    sorted_word_idx_count = sorted(word_idx_count.items(), key = operator.itemgetter(1),reverse = True)
    word2idx_small = {}
    new_idx = 0
    idx_new_idx_map = {}

    for idx,count in sorted_word_idx_count[:n_vocab]:
        word = idx2word[idx]
        print(word, count)
        word2idx_small[word] = new_idx
        idx_new_idx_map[idx] = new_idx
        new_idx +=1
    word2idx_small['UNKNOWN'] = new_idx
    unknown = new_idx
    assert('START' in word2idx_small)
    assert('END' in word2idx_small)
    for word in keep_words:
        assert(word in word2idx_small)
    sentences_small = []
    for sentence in indexed_sentences:
        if len(sentence) > 1:
            new_sentence = [idx_new_idx_map[idx] if idx in idx_new_idx_map else unknown for idx in sentence]
            sentences_small.append(new_sentence)
    return sentences_small, word2idx_small